In [2]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize, imsave
import scipy.io
from scipy.io import savemat, loadmat
from os import listdir, walk
from os.path import isfile, join
#import PIL
import matplotlib.pyplot as plt
import time

start_time = time.time()

trainPath = 'data/train/'
testPath = 'data/test'
avgImgHeight = int(64)       # Actual average height is 360. I am doing more downsampling
avgImgWidth = int(64)        # Actual average width is 400. I am doing more downsampling
imgClasses = ['dog', 'cat']
numClasses = len(imgClasses)
numTrain = len(next(walk(trainPath))[2])

def readData(path):
    fileNames = next(walk(path))[2]
    numFiles = len(fileNames)
    dataX = np.zeros((numFiles, avgImgHeight, avgImgWidth, 3), dtype=np.uint8)
    dataY = np.zeros((numFiles, numClasses), dtype=np.uint8)
    for i, fileName in enumerate(fileNames):
        img = imread(join(path, fileName))
        img = imresize(img, (avgImgHeight, avgImgWidth))
        dataX[i, :, :, :] = img

        for j, imgClass in enumerate(imgClasses):
            dataY[i, j] = (fileName[0:3] == imgClass)
    return dataX, dataY

trainDataX, trainDataY = readData(trainPath)
testDataX, testDataY = readData(testPath)

print('Time taken to load data = ', time.time()-start_time)

Time taken to load data =  126.56752490997314


In [3]:
learningRate = 1e-5
numIter = 10000
batchSize = 100

X = tf.placeholder(tf.float32, [None, avgImgHeight, avgImgWidth, 3])
y = tf.placeholder(tf.float32, [None, numClasses])


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


W_conv1 = weight_variable([5,5,3,32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([int(avgImgHeight/4 * avgImgWidth/4 * 64), 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, int(avgImgHeight/4 * avgImgWidth/4 * 64)])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, numClasses])
b_fc2 = bias_variable([numClasses])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()

sess.run(tf.initialize_all_variables())

for i in range(numIter):
    start_time = time.time()
    sampleIndices = np.random.choice(np.arange(numTrain), batchSize)
    batch_xs = trainDataX[sampleIndices]   #
    batch_ys = trainDataY[sampleIndices]
    sess.run(train_step, feed_dict={X: batch_xs, y: batch_ys, keep_prob: 0.5})
    duration = time.time() - start_time
    if i%10 == 0:
        print("Iteration = ", i, "Cross Entropy = ", sess.run(cross_entropy, feed_dict={X:batch_xs, y: batch_ys, keep_prob: 1.0}),
              "Train Accuracy = ", sess.run(accuracy, feed_dict={X:batch_xs, y: batch_ys, keep_prob: 1.0}), 
              "Validation Accuracy = ", sess.run(accuracy, feed_dict={X:testDataX[np.arange(batchSize)], y: testDataY[np.arange(batchSize)], keep_prob: 1.0}),
              "Duration = %.1f sec" % duration)

print("Final Train Accuracy = ", sess.run(accuracy, feed_dict={X: trainDataX, y: trainDataY, keep_prob: 1.0}))

Iteration =  0 Cross Entropy =  0.683064 Train Accuracy =  0.48 Validation Accuracy =  0.48 Duration = 0.9 sec
Iteration =  10 Cross Entropy =  0.704236 Train Accuracy =  0.47 Validation Accuracy =  0.52 Duration = 1.0 sec
Iteration =  20 Cross Entropy =  0.670978 Train Accuracy =  0.66 Validation Accuracy =  0.53 Duration = 0.9 sec
Iteration =  30 Cross Entropy =  0.672649 Train Accuracy =  0.6 Validation Accuracy =  0.57 Duration = 1.4 sec
Iteration =  40 Cross Entropy =  0.646082 Train Accuracy =  0.7 Validation Accuracy =  0.56 Duration = 1.4 sec
Iteration =  50 Cross Entropy =  0.671815 Train Accuracy =  0.53 Validation Accuracy =  0.55 Duration = 1.4 sec
Iteration =  60 Cross Entropy =  0.684318 Train Accuracy =  0.6 Validation Accuracy =  0.63 Duration = 1.4 sec
Iteration =  70 Cross Entropy =  0.650546 Train Accuracy =  0.63 Validation Accuracy =  0.61 Duration = 1.4 sec
Iteration =  80 Cross Entropy =  0.599862 Train Accuracy =  0.78 Validation Accuracy =  0.67 Duration = 1.4 

ResourceExhaustedError: OOM when allocating tensor with shape[22500,64,64,32]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_Placeholder_0, Variable/read)]]
Caused by op 'Conv2D', defined at:
  File "/home/robo/anaconda/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/robo/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/robo/.local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/robo/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/robo/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/robo/.local/lib/python3.5/site-packages/zmq/eventloop/io